In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import cohen_kappa_score
import copy
from sklearn.metrics import confusion_matrix
from features_general import aac_gen,dpc_gen,bin_aac
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import ExtraTreesClassifier 
import pickle
from sklearn.model_selection import cross_validate
import warnings
from sklearn.metrics import fbeta_score, make_scorer
import os
import sys
warnings.filterwarnings('ignore')

/home/krkd/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
def adjusted_classes(y_scores, t):
    return [1 if y >= t else -1 for y in y_scores]


In [4]:
def generate_thresh_files(X,Y,root,option,clf_array,kfold):
    path=root
    
    if option=='aac':
        path+='/aac_models/'
    elif option[0:3]=='aac':
        path+='/aac_models/'
    elif option=='dpc':
        path+='/dpc_models/'
    elif option[0:3]=='dpc':
        path+='/dpc_models/'
    elif option=='bin_n5':
        path+='/bin_n5_models/'
    elif option=='bin_n10':
        path+='/bin_n10_models/'
    elif option=='bin_n15':
        path+='/bin_n15_models/'
    elif option=='bin_c5':
        path+='/bin_c5_models/'
    elif option=='bin_c10':
        path+='/bin_c10_models/'
    elif option=='bin_c15':
        path+='/bin_c15_models/'
    elif option=='bin_nc5':
        path+='/bin_nc5_models/'
    elif option=='bin_nc10':
        path+='/bin_nc10_models/'
    elif option=='bin_nc15':
        path+='/bin_nc15_models/'
    
    filenames=["mlp","svm","knn","ridge","rf","extra_trees"]
    if kfold==1:
        filenames=["mlp_kfold","svm_kfold","knn_kfold","ridge_kfold","rf_kfold","extra_trees_kfold"]
    if len(clf_array)==1 and kfold==0:
        filenames=["svm"]
    if len(clf_array)==1 and kfold==1:
        filenames=["svm_kfold"]
            
    for i in range(len(clf_array)):
        fname=filenames[i]
        pathnow=path+fname+"_"+option+".txt"
        file=open(pathnow,"w")
        low=0
        high=1
        k=0.05
        clf=clf_array[i]
        print(clf)
        if hasattr(clf,'decision_function'):
            low=-3
            high=3
            k=0.1
        for threshold in np.arange(low,high,k):
            if kfold==0:
                Perform_testing(clf,fname,X,Y,threshold,file)
            elif kfold==1:
                Perform_KFold(clf,fname,X,Y,threshold,file)            
        file.close()


In [5]:
def load_models_run_test(option,root,X,Y):
    
    path=root
    flag=0
    if option=='aac':
        path+='/aac_models/'
    elif option[0:3]=='aac':
        flag=1
        path+='/aac_models/'
    elif option=='dpc':
        path+='/dpc_models/'
    elif option[0:3]=='dpc':
        flag=1
        path+='/dpc_models/'
    elif option=='bin_n5':
        path+='/bin_n5_models/'
    elif option=='bin_n10':
        path+='/bin_n10_models/'
    elif option=='bin_n15':
        path+='/bin_n15_models/'
    elif option=='bin_c5':
        path+='/bin_c5_models/'
    elif option=='bin_c10':
        path+='/bin_c10_models/'
    elif option=='bin_c15':
        path+='/bin_c15_models/'
    elif option=='bin_nc5':
        path+='/bin_nc5_models/'
    elif option=='bin_nc10':
        path+='/bin_nc10_models/'
    elif option=='bin_nc15':
        path+='/bin_nc15_models/'
    
    if flag==1:
        clf2 = pickle.load(open(path + 'svm_' + option + '.pickle','rb'))
        return [clf2.best_estimator_]
        
    #Load MLP
    clf1 = pickle.load(open(path + 'mlp_' + option + '.pickle','rb'))
    
    #Load SVM
    clf2 = pickle.load(open(path + 'svm_' + option + '.pickle','rb'))
    
    #Load KNN
    clf3 = pickle.load(open(path + 'knn_' + option + '.pickle','rb'))
    
    #Load ridge
    clf4 = pickle.load(open(path + 'ridge_' + option + '.pickle','rb'))
    
    #Load rf
    clf5 = pickle.load(open(path + 'rf_' + option + '.pickle','rb'))
    
    #Load extra trees
    clf6 = pickle.load(open(path + 'extra_trees_' + option + '.pickle','rb'))
    #Perform_testing(clf,'extra_trees',X,Y,t)
    return [clf1.best_estimator_,clf2.best_estimator_,clf3.best_estimator_,clf4.best_estimator_,clf5.best_estimator_,clf6.best_estimator_]
        
    

In [6]:
def Perform_KFold(clf,name,X,Y,t,file):
    kf = KFold(n_splits=5,shuffle=True,random_state=42)
    X_trainarr=[]
    Y_trainarr=[]
    
    X_testarr=[]
    Y_testarr=[]
    
    for train_index, test_index in kf.split(X):
        X_trainx, X_testx = getX(X,train_index),getX(X,test_index)
        Y_trainx, Y_testx = getX(Y,train_index),getX(Y,test_index)
        
        X_trainarr+=[X_trainx]
        Y_trainarr+=[Y_trainx]
        X_testarr+=[X_testx]
        Y_testarr+=[Y_testx]
        
    
    accuracies=[]
    specificities=[]
    senstivities=[]
    roc_auc_scores=[]
    mcc_scores=[]
    cohen_scores=[]
    cnt=0
    for i in range(5):
        X_train, X_test = X_trainarr[i],X_testarr[i]
        Y_train, Y_test = Y_trainarr[i],Y_testarr[i]
        clf.fit(X_train,Y_train)
        
        Y_scores=[]
        if hasattr(clf, "decision_function"):
            Y_scores=clf.decision_function(X_test)
        else:
            Y_scores=clf.predict_proba(X_test)[:,1]
            
        Y_pred = adjusted_classes(Y_scores,t)    
        
        acc_score=accuracy_score(Y_test,Y_pred)
        tn, fp, fn, tp = confusion_matrix(Y_test, Y_pred).ravel()
        specificity = tn / (tn+fp)
        senstivity = tp /(fn+tp)
        rocauc_score=roc_auc_score(Y_test, Y_scores)
        mcc_score=matthews_corrcoef(Y_test,Y_pred)  
        cohen_score = cohen_kappa_score(Y_test, Y_pred)
        #Matrices
        accuracies+=[acc_score]
        specificities+=[specificity]
        senstivities+=[senstivity]
        roc_auc_scores+=[rocauc_score]
        mcc_scores+=[mcc_score]
        cohen_scores+=[cohen_score]
    accuracies=np.asarray(accuracies)
    specificities=np.asarray(specificities)
    senstivities=np.asarray(senstivities)
    roc_auc_scores=np.asarray(roc_auc_scores)
    mcc_scores=np.asarray(mcc_scores)
    cohen_scores=np.asarray(cohen_scores)
    file.write("Threshold: %0.2f ," % (t))
    file.write("Senstivity: %0.2f ," % (senstivity))
    file.write("Specificity: %0.2f ," % (specificity))
    file.write("Accuracy: %0.2f ," % (acc_score))
    file.write("mcc scores: %0.2f ," % (mcc_score))
    file.write("roc-auc scores: %0.2f ," % (rocauc_score))
    file.write("cohen scores: %0.2f " % (cohen_score))
    file.write('\n')
    
    

In [7]:
def sensitivity(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp+fn)
    return sensitivity

def specificity(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn+fp)
    return specificity

scoring = {'rocauc': make_scorer(roc_auc_score),
           'accuracy': make_scorer(accuracy_score),
           'sensitivity': make_scorer(sensitivity), 
           'specificity': make_scorer(specificity),
           'mcc': make_scorer(matthews_corrcoef),
           'kappa': make_scorer(cohen_kappa_score)}

def print_results(cv_results, classifier):
    #print(classifier)
    print('Accuracy: ', "%.2f" % cv_results['test_accuracy'].mean(),end=',')  
    print('Specificity: ', "%.2f" % cv_results['test_specificity'].mean(),end=',')
    print('Sensitivity: ', "%.2f" % cv_results['test_sensitivity'].mean(),end=',')
    
    print('ROC: ', "%.2f" % cv_results['test_rocauc'].mean(),end=',')  
    print('MCC: ', "%.2f" % cv_results['test_mcc'].mean(),end=',')
    print('Cohen\'s Kappa: ', "%.2f" % cv_results['test_kappa'].mean(),end=',')


def generate_results(clf, features, labels, fold_size=5):
    cv_results = cross_validate(clf.fit(features, labels), features, labels, scoring=scoring, cv=fold_size,n_jobs=-1)
    print_results(cv_results, clf)

In [8]:
def getVector(line,option1,option2,x=None,y=None):
    if option1=='aac':
        return aac_gen(line,option2,x,y)
    elif option1=='dpc':
        return dpc_gen(line,option2,x,y)
    elif option1=='bin':
        return bin_aac(line,option2,x,y)

In [9]:
def getX(X,indexes):
    ans=[]
    for i in indexes:
        ans+=[X[i]]
    return ans

In [10]:
def getXYforfeature(name,option1,option2,append,train,x=None,y=None):
    lennow=1
    if x!=None and y!=None: 
        lennow=max(x,y)
    elif x!=None:
        lennow=x
    elif y!=None:
        lennow=y
    positive_file=""
    negative_file=""
    if train==0:
        positive_file=append+'/validation/pos_valid'
        negative_file=append+'/validation/neg_valid'
    else:
        positive_file=append+'/train/pos_train'
        negative_file=append+'/train/neg_train'
        
    X=[]
    Y=[]
    cnt=0
    f1=open(positive_file,'r')
    for line in f1:
        line=line.strip()
        line=line.upper()
        #print(line)
        if line.count('U')==0 and len(line)>=lennow and line.count(" ")==0:
            X+=[getVector(line,option1,option2,x,y)]
            Y+=[+1]
    f1.close()
    
    f2=open(negative_file,'r')
    for line in f2:
        line=line.strip()
        line=line.upper()
        if line.count('U')==0 and len(line)>=lennow:
            X+=[getVector(line,option1,option2,x,y)]
            Y+=[-1]
    f2.close()  
    return X,Y

In [11]:
def Perform_testing(clf,name,X,Y,t,file):
    Y_test=Y
    Y_pred=clf.predict(X)
    
    accuracies=[]
    specificities=[]
    senstivities=[]
    roc_auc_scores=[]
    mcc_scores=[]
    cohen_scores=[]
    
    Y_scores=[]
    if hasattr(clf,'decision_function'):
        Y_scores=clf.decision_function(X)
    else:
        Y_scores=clf.predict_proba(X)[:,1]
    
    Y_pred = adjusted_classes(Y_scores,t)   
    
    acc_score=accuracy_score(Y_test,Y_pred)
    tn,fp,fn,tp = confusion_matrix(Y_test, Y_pred).ravel()
    specificity = tn / (tn+fp)
    senstivity = tp /(fn+tp)
    rocauc_score=roc_auc_score(Y_test, Y_scores)
    mcc_score=matthews_corrcoef(Y_test,Y_pred)  
    cohen_score = cohen_kappa_score(Y_test, Y_pred)
    
    
    file.write("Threshold: %0.2f ," % (t))
    file.write("Senstivity: %0.2f ," % (senstivity))
    file.write("Specificity: %0.2f ," % (specificity))
    file.write("Accuracy: %0.2f ," % (acc_score))
    file.write("mcc scores: %0.2f ," % (mcc_score))
    file.write("roc-auc scores: %0.2f ," % (rocauc_score))
    file.write("cohen scores: %0.2f " % (cohen_score))
    file.write('\n')

In [12]:
root1='./ACPs and non-ACPS'



In [13]:
#AAC Normal--testing
print('AAC')
X_train_aac,Y_train_aac=getXYforfeature('aac','aac','Normal',root1,1)
X_aac,Y_aac=getXYforfeature('aac','aac','Normal',root1,0)
clf_array=load_models_run_test('aac',root1,X_aac,Y_aac)
generate_thresh_files(X_aac,Y_aac,root1,'aac',clf_array,0)
generate_thresh_files(X_aac,Y_aac,root1,'aac',clf_array,1)
print("_________________________________")


#AAC N5--testing
print('AAC_N5')
X_train_aac_n5,Y_train_aac_n5=getXYforfeature('aac','aac','N',root1,1,x=5)
X_aac_n5,Y_aac_n5=getXYforfeature('aac','aac','N',root1,0,x=5)
clf_array=load_models_run_test('aac_n5',root1,X_aac_n5,Y_aac_n5)
generate_thresh_files(X_aac_n5,Y_aac_n5,root1,'aac_n5',clf_array,0)
generate_thresh_files(X_aac_n5,Y_aac_n5,root1,'aac_n5',clf_array,1)
print("_________________________________")



#AAC N10--testing
print('AAC_N10')
X_train_aac_n10,Y_train_aac_n10=getXYforfeature('aac','aac','N',root1,1,x=10)
X_aac_n10,Y_aac_n10=getXYforfeature('aac','aac','N',root1,0,x=10)
clf_array=load_models_run_test('aac_n10',root1,X_aac_n10,Y_aac_n10)
generate_thresh_files(X_aac_n10,Y_aac_n10,root1,'aac_n10',clf_array,0)
generate_thresh_files(X_aac_n10,Y_aac_n10,root1,'aac_n10',clf_array,1)
print("_________________________________")


#AAC 1--testing
print('AAC_N15')
X_train_aac_n15,Y_train_aac_n15=getXYforfeature('aac','aac','N',root1,1,x=15)
X_aac_n15,Y_aac_n15=getXYforfeature('aac','aac','N',root1,0,x=15)
clf_array=load_models_run_test('aac_n15',root1,X_aac_n15,Y_aac_n15)
generate_thresh_files(X_aac_n15,Y_aac_n15,root1,'aac_n15',clf_array,0)
generate_thresh_files(X_aac_n15,Y_aac_n15,root1,'aac_n15',clf_array,1)
print("_________________________________")




#AAC 1--testing
print('AAC_C5')
X_train_aac_c5,Y_train_aac_c5=getXYforfeature('aac','aac','C',root1,1,x=5)
X_aac_c5,Y_aac_c5=getXYforfeature('aac','aac','C',root1,0,x=5)
clf_array=load_models_run_test('aac_c5',root1,X_aac_c5,Y_aac_c5)
generate_thresh_files(X_aac_c5,Y_aac_c5,root1,'aac_c5',clf_array,0)
generate_thresh_files(X_aac_c5,Y_aac_c5,root1,'aac_c5',clf_array,1)
print("_________________________________")



#AAC 1--testing
print('AAC_C10')
X_train_aac_c10,Y_train_aac_c10=getXYforfeature('aac','aac','C',root1,1,x=10)
X_aac_c10,Y_aac_c10=getXYforfeature('aac','aac','C',root1,0,x=10)
clf_array=load_models_run_test('aac_c10',root1,X_aac_c10,Y_aac_c10)
generate_thresh_files(X_aac_c10,Y_aac_c10,root1,'aac_c10',clf_array,0)
generate_thresh_files(X_aac_c10,Y_aac_c10,root1,'aac_c10',clf_array,1)
print("_________________________________")


#AAC 1--testing
print('AAC_C15')
X_train_aac_c15,Y_train_aac_c15=getXYforfeature('aac','aac','C',root1,1,x=15)
X_aac_c15,Y_aac_c15=getXYforfeature('aac','aac','C',root1,0,x=15)
clf_array=load_models_run_test('aac_c15',root1,X_aac_c15,Y_aac_c15)
generate_thresh_files(X_aac_c15,Y_aac_c15,root1,'aac_c15',clf_array,0)
generate_thresh_files(X_aac_c15,Y_aac_c15,root1,'aac_c15',clf_array,1)
print("_________________________________")




#AAC 1--testing
print('AAC_NC5')
X_train_aac_nc5,Y_train_aac_nc5=getXYforfeature('aac','aac','NC',root1,1,x=5,y=5)
X_aac_nc5,Y_aac_nc5=getXYforfeature('aac','aac','NC',root1,0,x=5,y=5)
clf_array=load_models_run_test('aac_nc5',root1,X_aac_nc5,Y_aac_nc5)
generate_thresh_files(X_aac_nc5,Y_aac_nc5,root1,'aac_nc5',clf_array,0)
generate_thresh_files(X_aac_nc5,Y_aac_nc5,root1,'aac_nc5',clf_array,1)
print("_________________________________")



#AAC 1--testing
print('AAC_NC10')
X_train_aac_nc10,Y_train_aac_nc10=getXYforfeature('aac','aac','NC',root1,1,x=10,y=10)
X_aac_nc10,Y_aac_nc10=getXYforfeature('aac','aac','NC',root1,0,x=10,y=10)
clf_array=load_models_run_test('aac_nc10',root1,X_aac_nc10,Y_aac_nc10)
generate_thresh_files(X_aac_nc10,Y_aac_nc10,root1,'aac_nc10',clf_array,0)
generate_thresh_files(X_aac_nc10,Y_aac_nc10,root1,'aac_nc10',clf_array,1)
print("_________________________________")


#AAC 1--testing
print('AAC_NC15')
X_train_aac_nc15,Y_train_aac_nc15=getXYforfeature('aac','aac','NC',root1,1,x=15,y=15)
X_aac_nc15,Y_aac_nc15=getXYforfeature('aac','aac','NC',root1,0,x=15,y=15)
clf_array=load_models_run_test('aac_nc15',root1,X_aac_nc15,Y_aac_nc15)
generate_thresh_files(X_aac_nc15,Y_aac_nc15,root1,'aac_nc15',clf_array,0)
generate_thresh_files(X_aac_nc15,Y_aac_nc15,root1,'aac_nc15',clf_array,1)
print("_________________________________")



AAC
MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)
SVC(C=2, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
           metric_params=None, n_jobs=1, n_neighbors=13, p=2,
           weights='uniform')
RidgeClassifier(alpha=20.0, class_weight=None, copy_X=True,
        fit_intercept=True, max_iter=None, normalize=False,
        random_state=None, solver='auto', tol=0.0001)


In [14]:
# #DPC 1--testing
print('DPC')
X_train_dpc,Y_train_dpc=getXYforfeature('dpc','dpc','Normal',root1,1)
X_dpc,Y_dpc=getXYforfeature('dpc','dpc','Normal',root1,0)
clf_array=load_models_run_test('dpc',root1,X_dpc,Y_dpc)
generate_thresh_files(X_dpc,Y_dpc,root1,'dpc',clf_array,0)
generate_thresh_files(X_dpc,Y_dpc,root1,'dpc',clf_array,1)
print("_________________________________")


#DPC N5--testing
print('DPC_N5')
X_train_dpc_n5,Y_train_dpc_n5=getXYforfeature('dpc','dpc','N',root1,1,x=5)
X_dpc_n5,Y_dpc_n5=getXYforfeature('dpc','dpc','N',root1,0,x=5)
clf_array=load_models_run_test('dpc_n5',root1,X_dpc_n5,Y_dpc_n5)
generate_thresh_files(X_dpc_n5,Y_dpc_n5,root1,'dpc_n5',clf_array,0)
generate_thresh_files(X_dpc_n5,Y_dpc_n5,root1,'dpc_n5',clf_array,1)


print("_________________________________")


#DPC N10--testing
print('DPC_N10')
X_train_dpc_n10,Y_train_dpc_n10=getXYforfeature('dpc','dpc','N',root1,1,x=10)
X_dpc_n10,Y_dpc_n10=getXYforfeature('dpc','dpc','N',root1,0,x=10)
clf_array=load_models_run_test('dpc_n10',root1,X_dpc_n10,Y_dpc_n10)
generate_thresh_files(X_dpc_n10,Y_dpc_n10,root1,'dpc_n10',clf_array,0)
generate_thresh_files(X_dpc_n10,Y_dpc_n10,root1,'dpc_n10',clf_array,1)
print("_________________________________")

#DPC N15--testing
print('DPC_N15')
X_train_dpc_n15,Y_train_dpc_n15=getXYforfeature('dpc','dpc','N',root1,1,x=15)
X_dpc_n15,Y_dpc_n15=getXYforfeature('dpc','dpc','N',root1,0,x=15)
clf_array=load_models_run_test('dpc_n15',root1,X_dpc_n15,Y_dpc_n15)
generate_thresh_files(X_dpc_n15,Y_dpc_n15,root1,'dpc_n15',clf_array,0)
generate_thresh_files(X_dpc_n15,Y_dpc_n15,root1,'dpc_n15',clf_array,1)

print("_________________________________")




#DPC C5--testing
print('DPC_C5')
X_train_dpc_c5,Y_train_dpc_c5=getXYforfeature('dpc','dpc','C',root1,1,x=5)
X_dpc_c5,Y_dpc_c5=getXYforfeature('dpc','dpc','C',root1,0,x=5)
clf_array=load_models_run_test('dpc_c5',root1,X_dpc_c5,Y_dpc_c5)
generate_thresh_files(X_dpc_c5,Y_dpc_c5,root1,'dpc_c5',clf_array,0)
generate_thresh_files(X_dpc_c5,Y_dpc_c5,root1,'dpc_c5',clf_array,1)
print("_________________________________")


#DPC C10--testing
print('DPC_C10')
X_train_dpc_c10,Y_train_dpc_c10=getXYforfeature('dpc','dpc','C',root1,1,x=10)
X_dpc_c10,Y_dpc_c10=getXYforfeature('dpc','dpc','C',root1,0,x=10)
clf_array=load_models_run_test('dpc_c10',root1,X_dpc_c10,Y_dpc_c10)
generate_thresh_files(X_dpc_c10,Y_dpc_c10,root1,'dpc_c10',clf_array,0)
generate_thresh_files(X_dpc_c10,Y_dpc_c10,root1,'dpc_c10',clf_array,1)
print("_________________________________")

#DPC C15--testing
print('DPC_C15')
X_train_dpc_c15,Y_train_dpc_c15=getXYforfeature('dpc','dpc','C',root1,1,x=15)
X_dpc_c15,Y_dpc_c15=getXYforfeature('dpc','dpc','C',root1,0,x=15)
clf_array=load_models_run_test('dpc_c15',root1,X_dpc_c15,Y_dpc_c15)
generate_thresh_files(X_dpc_c15,Y_dpc_c15,root1,'dpc_c15',clf_array,0)
generate_thresh_files(X_dpc_c15,Y_dpc_c15,root1,'dpc_c15',clf_array,1)
print("_________________________________")






#DPC NC5--testing
print('DPC_NC5')
X_train_dpc_nc5,Y_train_dpc_nc5=getXYforfeature('dpc','dpc','NC',root1,1,x=5,y=5)
X_dpc_nc5,Y_dpc_nc5=getXYforfeature('dpc','dpc','NC',root1,0,x=5,y=5)
clf_array=load_models_run_test('dpc_nc5',root1,X_dpc_nc5,Y_dpc_nc5)
generate_thresh_files(X_dpc_nc5,Y_dpc_nc5,root1,'dpc_nc5',clf_array,0)
generate_thresh_files(X_dpc_nc5,Y_dpc_nc5,root1,'dpc_nc5',clf_array,1)
print("_________________________________")


#DPC NC10--testing
print('DPC_NC10')
X_train_dpc_nc10,Y_train_dpc_nc10=getXYforfeature('dpc','dpc','NC',root1,1,x=10,y=10)
X_dpc_nc10,Y_dpc_nc10=getXYforfeature('dpc','dpc','NC',root1,0,x=10,y=10)
clf_array=load_models_run_test('dpc_nc10',root1,X_dpc_nc10,Y_dpc_nc10)
generate_thresh_files(X_dpc_nc10,Y_dpc_nc10,root1,'dpc_nc10',clf_array,0)
generate_thresh_files(X_dpc_nc10,Y_dpc_nc10,root1,'dpc_nc10',clf_array,1)
print("_________________________________")

#DPC NC15--testing
print('DPC_NC15')
X_train_dpc_nc15,Y_train_dpc_nc15=getXYforfeature('dpc','dpc','NC',root1,1,x=15,y=15)
X_dpc_nc15,Y_dpc_nc15=getXYforfeature('dpc','dpc','NC',root1,0,x=15,y=15)
clf_array=load_models_run_test('dpc_nc15',root1,X_dpc_nc15,Y_dpc_nc15)
generate_thresh_files(X_dpc_nc15,Y_dpc_nc15,root1,'dpc_nc15',clf_array,0)
generate_thresh_files(X_dpc_nc15,Y_dpc_nc15,root1,'dpc_nc15',clf_array,1)
print("_________________________________")






DPC
MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='sgd', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)
SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')
RidgeClassifier(alpha=5.0, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, random_state=None, solver='auto',
        tol=0.0001)
Ran

In [15]:
#N5 1--testing
print('N5')
X_train_bin_n5,Y_train_bin_n5=getXYforfeature('bin','bin','N',root1,1,x=5)
X_bin_n5,Y_bin_n5=getXYforfeature('bin','bin','N',root1,0,x=5)
clf_array=load_models_run_test('bin_n5',root1,X_bin_n5,Y_bin_n5)
generate_thresh_files(X_bin_n5,Y_bin_n5,root1,'bin_n5',clf_array,0)
generate_thresh_files(X_bin_n5,Y_bin_n5,root1,'bin_n5',clf_array,1)
print("_________________________________")




#N10 1--testing
print('N10')
X_train_bin_n10,Y_train_bin_n10=getXYforfeature('bin','bin','N',root1,1,x=10)
X_bin_n10,Y_bin_n10=getXYforfeature('bin','bin','N',root1,0,x=10)
clf_array=load_models_run_test('bin_n10',root1,X_bin_n10,Y_bin_n10)
generate_thresh_files(X_bin_n10,Y_bin_n10,root1,'bin_n10',clf_array,0)
generate_thresh_files(X_bin_n10,Y_bin_n10,root1,'bin_n10',clf_array,1)
print("_________________________________")



#N15 1--testing
print('N15')
X_train_bin_n15,Y_train_bin_n15=getXYforfeature('bin','bin','N',root1,1,x=15)
X_bin_n15,Y_bin_n15=getXYforfeature('bin','bin','N',root1,0,x=15)
clf_array=load_models_run_test('bin_n15',root1,X_bin_n15,Y_bin_n15)
generate_thresh_files(X_bin_n15,Y_bin_n15,root1,'bin_n15',clf_array,0)
generate_thresh_files(X_bin_n15,Y_bin_n15,root1,'bin_n15',clf_array,1)
print("_________________________________")

#C5 1--testing
print('C5')
X_train_bin_c5,Y_train_bin_c5=getXYforfeature('bin','bin','C',root1,1,x=5)
X_bin_c5,Y_bin_c5=getXYforfeature('bin','bin','C',root1,0,x=5)
clf_array=load_models_run_test('bin_c5',root1,X_bin_c5,Y_bin_c5)
generate_thresh_files(X_bin_c5,Y_bin_c5,root1,'bin_c5',clf_array,0)
generate_thresh_files(X_bin_c5,Y_bin_c5,root1,'bin_c5',clf_array,1)
print("_________________________________")

#C10 1--testing
print('C10')
X_train_bin_c10,Y_train_bin_c10=getXYforfeature('bin','bin','C',root1,1,x=10)
X_bin_c10,Y_bin_c10=getXYforfeature('bin','bin','C',root1,0,x=10)
clf_array=load_models_run_test('bin_c10',root1,X_bin_c10,Y_bin_c10)
generate_thresh_files(X_bin_c10,Y_bin_c10,root1,'bin_c10',clf_array,0)
generate_thresh_files(X_bin_c10,Y_bin_c10,root1,'bin_c10',clf_array,1)
print("_________________________________")


#C15 1--testing
print('C15')
X_train_bin_c15,Y_train_bin_c15=getXYforfeature('bin','bin','C',root1,1,x=15)
X_bin_c15,Y_bin_c15=getXYforfeature('bin','bin','C',root1,0,x=15)
clf_array=load_models_run_test('bin_c15',root1,X_bin_c15,Y_bin_c15)
generate_thresh_files(X_bin_c15,Y_bin_c15,root1,'bin_c15',clf_array,0)
generate_thresh_files(X_bin_c15,Y_bin_c15,root1,'bin_c15',clf_array,1)
print("_________________________________")


#N5C5 1--testing
print('N5C5')
X_train_bin_nc5,Y_train_bin_nc5=getXYforfeature('bin','bin','NC',root1,1,x=5,y=5)
X_bin_nc5,Y_bin_nc5=getXYforfeature('bin','bin','NC',root1,0,x=5,y=5)
clf_array=load_models_run_test('bin_nc5',root1,X_bin_nc5,Y_bin_nc5)
generate_thresh_files(X_bin_nc5,Y_bin_nc5,root1,'bin_nc5',clf_array,0)
generate_thresh_files(X_bin_nc5,Y_bin_nc5,root1,'bin_nc5',clf_array,1)
print("_________________________________")




#N10C10 1--testing
print('N10C10')
X_train_bin_nc10,Y_train_bin_nc10=getXYforfeature('bin','bin','NC',root1,1,x=10,y=10)
X_bin_nc10,Y_bin_nc10=getXYforfeature('bin','bin','NC',root1,0,x=10,y=10)
clf_array=load_models_run_test('bin_nc10',root1,X_bin_nc10,Y_bin_nc10)
generate_thresh_files(X_bin_nc10,Y_bin_nc10,root1,'bin_nc10',clf_array,0)
generate_thresh_files(X_bin_nc10,Y_bin_nc10,root1,'bin_nc10',clf_array,1)
print("_________________________________")

#N15C15 1--testing
print('N15C15')
X_train_bin_nc15,Y_train_bin_nc15=getXYforfeature('bin','bin','NC',root1,1,x=15,y=15)
X_bin_nc15,Y_bin_nc15=getXYforfeature('bin','bin','NC',root1,0,x=15,y=15)
clf_array=load_models_run_test('bin_nc15',root1,X_bin_nc15,Y_bin_nc15)
generate_thresh_files(X_bin_nc15,Y_bin_nc15,root1,'bin_nc15',clf_array,0)
generate_thresh_files(X_bin_nc15,Y_bin_nc15,root1,'bin_nc15',clf_array,1)
print("_________________________________")





N5
MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)
SVC(C=2, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')
RidgeClassifier(alpha=2.0, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, random_state=None, solver='auto',
        tol=0.0001)
Rando

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=16, max_features=6, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=11, max_features=16, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=20,
           min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 10), learning_rate='

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')
RidgeClassifier(alpha=50.0, class_weight=None, copy_X=True,
        fit_intercept=True, max_iter=None, normalize=False,
        random_state=None, solver='auto', tol=0.0001)
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=11, max_features=16, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=20,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
ExtraTreesClassifier(bootst

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=11, max_features=6, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=10,
           min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
_________________________________
N10C10
MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='sgd', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)
SVC(C=2, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  ma

In [16]:
root1='./ACPs and random peptides'

In [17]:
#AAC Normal--testing
print('AAC')
X_train_aac,Y_train_aac=getXYforfeature('aac','aac','Normal',root1,1)
X_aac,Y_aac=getXYforfeature('aac','aac','Normal',root1,0)
clf_array=load_models_run_test('aac',root1,X_aac,Y_aac)
generate_thresh_files(X_aac,Y_aac,root1,'aac',clf_array,0)
generate_thresh_files(X_aac,Y_aac,root1,'aac',clf_array,1)
print("_________________________________")


#AAC N5--testing
print('AAC_N5')
X_train_aac_n5,Y_train_aac_n5=getXYforfeature('aac','aac','N',root1,1,x=5)
X_aac_n5,Y_aac_n5=getXYforfeature('aac','aac','N',root1,0,x=5)
clf_array=load_models_run_test('aac_n5',root1,X_aac_n5,Y_aac_n5)
generate_thresh_files(X_aac_n5,Y_aac_n5,root1,'aac_n5',clf_array,0)
generate_thresh_files(X_aac_n5,Y_aac_n5,root1,'aac_n5',clf_array,1)
print("_________________________________")



#AAC N10--testing
print('AAC_N10')
X_train_aac_n10,Y_train_aac_n10=getXYforfeature('aac','aac','N',root1,1,x=10)
X_aac_n10,Y_aac_n10=getXYforfeature('aac','aac','N',root1,0,x=10)
clf_array=load_models_run_test('aac_n10',root1,X_aac_n10,Y_aac_n10)
generate_thresh_files(X_aac_n10,Y_aac_n10,root1,'aac_n10',clf_array,0)
generate_thresh_files(X_aac_n10,Y_aac_n10,root1,'aac_n10',clf_array,1)
print("_________________________________")


#AAC 1--testing
print('AAC_N15')
X_train_aac_n15,Y_train_aac_n15=getXYforfeature('aac','aac','N',root1,1,x=15)
X_aac_n15,Y_aac_n15=getXYforfeature('aac','aac','N',root1,0,x=15)
clf_array=load_models_run_test('aac_n15',root1,X_aac_n15,Y_aac_n15)
generate_thresh_files(X_aac_n15,Y_aac_n15,root1,'aac_n15',clf_array,0)
generate_thresh_files(X_aac_n15,Y_aac_n15,root1,'aac_n15',clf_array,1)
print("_________________________________")




#AAC 1--testing
print('AAC_C5')
X_train_aac_c5,Y_train_aac_c5=getXYforfeature('aac','aac','C',root1,1,x=5)
X_aac_c5,Y_aac_c5=getXYforfeature('aac','aac','C',root1,0,x=5)
clf_array=load_models_run_test('aac_c5',root1,X_aac_c5,Y_aac_c5)
generate_thresh_files(X_aac_c5,Y_aac_c5,root1,'aac_c5',clf_array,0)
generate_thresh_files(X_aac_c5,Y_aac_c5,root1,'aac_c5',clf_array,1)
print("_________________________________")



#AAC 1--testing
print('AAC_C10')
X_train_aac_c10,Y_train_aac_c10=getXYforfeature('aac','aac','C',root1,1,x=10)
X_aac_c10,Y_aac_c10=getXYforfeature('aac','aac','C',root1,0,x=10)
clf_array=load_models_run_test('aac_c10',root1,X_aac_c10,Y_aac_c10)
generate_thresh_files(X_aac_c10,Y_aac_c10,root1,'aac_c10',clf_array,0)
generate_thresh_files(X_aac_c10,Y_aac_c10,root1,'aac_c10',clf_array,1)
print("_________________________________")


#AAC 1--testing
print('AAC_C15')
X_train_aac_c15,Y_train_aac_c15=getXYforfeature('aac','aac','C',root1,1,x=15)
X_aac_c15,Y_aac_c15=getXYforfeature('aac','aac','C',root1,0,x=15)
clf_array=load_models_run_test('aac_c15',root1,X_aac_c15,Y_aac_c15)
generate_thresh_files(X_aac_c15,Y_aac_c15,root1,'aac_c15',clf_array,0)
generate_thresh_files(X_aac_c15,Y_aac_c15,root1,'aac_c15',clf_array,1)
print("_________________________________")




#AAC 1--testing
print('AAC_NC5')
X_train_aac_nc5,Y_train_aac_nc5=getXYforfeature('aac','aac','NC',root1,1,x=5,y=5)
X_aac_nc5,Y_aac_nc5=getXYforfeature('aac','aac','NC',root1,0,x=5,y=5)
clf_array=load_models_run_test('aac_nc5',root1,X_aac_nc5,Y_aac_nc5)
generate_thresh_files(X_aac_nc5,Y_aac_nc5,root1,'aac_nc5',clf_array,0)
generate_thresh_files(X_aac_nc5,Y_aac_nc5,root1,'aac_nc5',clf_array,1)
print("_________________________________")



#AAC 1--testing
print('AAC_NC10')
X_train_aac_nc10,Y_train_aac_nc10=getXYforfeature('aac','aac','NC',root1,1,x=10,y=10)
X_aac_nc10,Y_aac_nc10=getXYforfeature('aac','aac','NC',root1,0,x=10,y=10)
clf_array=load_models_run_test('aac_nc10',root1,X_aac_nc10,Y_aac_nc10)
generate_thresh_files(X_aac_nc10,Y_aac_nc10,root1,'aac_nc10',clf_array,0)
generate_thresh_files(X_aac_nc10,Y_aac_nc10,root1,'aac_nc10',clf_array,1)
print("_________________________________")


#AAC 1--testing
print('AAC_NC15')
X_train_aac_nc15,Y_train_aac_nc15=getXYforfeature('aac','aac','NC',root1,1,x=15,y=15)
X_aac_nc15,Y_aac_nc15=getXYforfeature('aac','aac','NC',root1,0,x=15,y=15)
clf_array=load_models_run_test('aac_nc15',root1,X_aac_nc15,Y_aac_nc15)
generate_thresh_files(X_aac_nc15,Y_aac_nc15,root1,'aac_nc15',clf_array,0)
generate_thresh_files(X_aac_nc10,Y_aac_nc10,root1,'aac_nc15',clf_array,1)
print("_________________________________")



AAC
MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='sgd', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)
SVC(C=3, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
           metric_params=None, n_jobs=1, n_neighbors=9, p=2,
           weights='uniform')
RidgeClassifier(alpha=50.0, class_weight=None, copy_X=True,
        fit_intercept=True, max_iter=None, normalize=False,
        random_state=None, solver='auto', tol=0.0001)
Ra

In [18]:
# #DPC 1--testing
print('DPC')
X_train_dpc,Y_train_dpc=getXYforfeature('dpc','dpc','Normal',root1,1)
X_dpc,Y_dpc=getXYforfeature('dpc','dpc','Normal',root1,0)
clf_array=load_models_run_test('dpc',root1,X_dpc,Y_dpc)
generate_thresh_files(X_dpc,Y_dpc,root1,'dpc',clf_array,0)
generate_thresh_files(X_dpc,Y_dpc,root1,'dpc',clf_array,1)
print("_________________________________")


#DPC N5--testing
print('DPC_N5')
X_train_dpc_n5,Y_train_dpc_n5=getXYforfeature('dpc','dpc','N',root1,1,x=5)
X_dpc_n5,Y_dpc_n5=getXYforfeature('dpc','dpc','N',root1,0,x=5)
clf_array=load_models_run_test('dpc_n5',root1,X_dpc_n5,Y_dpc_n5)
generate_thresh_files(X_dpc_n5,Y_dpc_n5,root1,'dpc_n5',clf_array,0)
generate_thresh_files(X_dpc_n5,Y_dpc_n5,root1,'dpc_n5',clf_array,1)


print("_________________________________")


#DPC N10--testing
print('DPC_N10')
X_train_dpc_n10,Y_train_dpc_n10=getXYforfeature('dpc','dpc','N',root1,1,x=10)
X_dpc_n10,Y_dpc_n10=getXYforfeature('dpc','dpc','N',root1,0,x=10)
clf_array=load_models_run_test('dpc_n10',root1,X_dpc_n10,Y_dpc_n10)
generate_thresh_files(X_dpc_n10,Y_dpc_n10,root1,'dpc_n10',clf_array,0)
generate_thresh_files(X_dpc_n10,Y_dpc_n10,root1,'dpc_n10',clf_array,1)
print("_________________________________")

#DPC N15--testing
print('DPC_N15')
X_train_dpc_n15,Y_train_dpc_n15=getXYforfeature('dpc','dpc','N',root1,1,x=15)
X_dpc_n15,Y_dpc_n15=getXYforfeature('dpc','dpc','N',root1,0,x=15)
clf_array=load_models_run_test('dpc_n15',root1,X_dpc_n15,Y_dpc_n15)
generate_thresh_files(X_dpc_n15,Y_dpc_n15,root1,'dpc_n15',clf_array,0)
generate_thresh_files(X_dpc_n15,Y_dpc_n15,root1,'dpc_n15',clf_array,1)

print("_________________________________")




#DPC C5--testing
print('DPC_C5')
X_train_dpc_c5,Y_train_dpc_c5=getXYforfeature('dpc','dpc','C',root1,1,x=5)
X_dpc_c5,Y_dpc_c5=getXYforfeature('dpc','dpc','C',root1,0,x=5)
clf_array=load_models_run_test('dpc_c5',root1,X_dpc_c5,Y_dpc_c5)
generate_thresh_files(X_dpc_c5,Y_dpc_c5,root1,'dpc_c5',clf_array,0)
generate_thresh_files(X_dpc_c5,Y_dpc_c5,root1,'dpc_c5',clf_array,1)
print("_________________________________")


#DPC C10--testing
print('DPC_C10')
X_train_dpc_c10,Y_train_dpc_c10=getXYforfeature('dpc','dpc','C',root1,1,x=10)
X_dpc_c10,Y_dpc_c10=getXYforfeature('dpc','dpc','C',root1,0,x=10)
clf_array=load_models_run_test('dpc_c10',root1,X_dpc_c10,Y_dpc_c10)
generate_thresh_files(X_dpc_c10,Y_dpc_c10,root1,'dpc_c10',clf_array,0)
generate_thresh_files(X_dpc_c10,Y_dpc_c10,root1,'dpc_c10',clf_array,1)
print("_________________________________")

#DPC C15--testing
print('DPC_C15')
X_train_dpc_c15,Y_train_dpc_c15=getXYforfeature('dpc','dpc','C',root1,1,x=15)
X_dpc_c15,Y_dpc_c15=getXYforfeature('dpc','dpc','C',root1,0,x=15)
clf_array=load_models_run_test('dpc_c15',root1,X_dpc_c15,Y_dpc_c15)
generate_thresh_files(X_dpc_c15,Y_dpc_c15,root1,'dpc_c15',clf_array,0)
generate_thresh_files(X_dpc_c15,Y_dpc_c15,root1,'dpc_c15',clf_array,1)
print("_________________________________")






#DPC NC5--testing
print('DPC_NC5')
X_train_dpc_nc5,Y_train_dpc_nc5=getXYforfeature('dpc','dpc','NC',root1,1,x=5,y=5)
X_dpc_nc5,Y_dpc_nc5=getXYforfeature('dpc','dpc','NC',root1,0,x=5,y=5)
clf_array=load_models_run_test('dpc_nc5',root1,X_dpc_nc5,Y_dpc_nc5)
generate_thresh_files(X_dpc_nc5,Y_dpc_nc5,root1,'dpc_nc5',clf_array,0)
generate_thresh_files(X_dpc_nc5,Y_dpc_nc5,root1,'dpc_nc5',clf_array,1)
print("_________________________________")


#DPC NC10--testing
print('DPC_NC10')
X_train_dpc_nc10,Y_train_dpc_nc10=getXYforfeature('dpc','dpc','NC',root1,1,x=10,y=10)
X_dpc_nc10,Y_dpc_nc10=getXYforfeature('dpc','dpc','NC',root1,0,x=10,y=10)
clf_array=load_models_run_test('dpc_nc10',root1,X_dpc_nc10,Y_dpc_nc10)
generate_thresh_files(X_dpc_nc10,Y_dpc_nc10,root1,'dpc_nc10',clf_array,0)
generate_thresh_files(X_dpc_nc10,Y_dpc_nc10,root1,'dpc_nc10',clf_array,1)
print("_________________________________")

#DPC NC15--testing
print('DPC_NC15')
X_train_dpc_nc15,Y_train_dpc_nc15=getXYforfeature('dpc','dpc','NC',root1,1,x=15,y=15)
X_dpc_nc15,Y_dpc_nc15=getXYforfeature('dpc','dpc','NC',root1,0,x=15,y=15)
clf_array=load_models_run_test('dpc_nc15',root1,X_dpc_nc15,Y_dpc_nc15)
generate_thresh_files(X_dpc_nc15,Y_dpc_nc15,root1,'dpc_nc15',clf_array,0)
generate_thresh_files(X_dpc_nc15,Y_dpc_nc15,root1,'dpc_nc15',clf_array,1)
print("_________________________________")






DPC
MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='sgd', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)
SVC(C=2, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')
RidgeClassifier(alpha=50.0, class_weight=None, copy_X=True,
        fit_intercept=True, max_iter=None, normalize=False,
        random_state=None, solver='auto', tol=0.0001)
Ra

In [19]:
#N5 1--testing
print('N5')
X_train_bin_n5,Y_train_bin_n5=getXYforfeature('bin','bin','N',root1,1,x=5)
X_bin_n5,Y_bin_n5=getXYforfeature('bin','bin','N',root1,0,x=5)
clf_array=load_models_run_test('bin_n5',root1,X_bin_n5,Y_bin_n5)
generate_thresh_files(X_bin_n5,Y_bin_n5,root1,'bin_n5',clf_array,0)
generate_thresh_files(X_bin_n5,Y_bin_n5,root1,'bin_n5',clf_array,1)
print("_________________________________")




#N10 1--testing
print('N10')
X_train_bin_n10,Y_train_bin_n10=getXYforfeature('bin','bin','N',root1,1,x=10)
X_bin_n10,Y_bin_n10=getXYforfeature('bin','bin','N',root1,0,x=10)
clf_array=load_models_run_test('bin_n10',root1,X_bin_n10,Y_bin_n10)
generate_thresh_files(X_bin_n10,Y_bin_n10,root1,'bin_n10',clf_array,0)
generate_thresh_files(X_bin_n10,Y_bin_n10,root1,'bin_n10',clf_array,1)
print("_________________________________")



#N15 1--testing
print('N15')
X_train_bin_n15,Y_train_bin_n15=getXYforfeature('bin','bin','N',root1,1,x=15)
X_bin_n15,Y_bin_n15=getXYforfeature('bin','bin','N',root1,0,x=15)
clf_array=load_models_run_test('bin_n15',root1,X_bin_n15,Y_bin_n15)
generate_thresh_files(X_bin_n15,Y_bin_n15,root1,'bin_n15',clf_array,0)
generate_thresh_files(X_bin_n15,Y_bin_n15,root1,'bin_n15',clf_array,1)
print("_________________________________")

#C5 1--testing
print('C5')
X_train_bin_c5,Y_train_bin_c5=getXYforfeature('bin','bin','C',root1,1,x=5)
X_bin_c5,Y_bin_c5=getXYforfeature('bin','bin','C',root1,0,x=5)
clf_array=load_models_run_test('bin_c5',root1,X_bin_c5,Y_bin_c5)
generate_thresh_files(X_bin_c5,Y_bin_c5,root1,'bin_c5',clf_array,0)
generate_thresh_files(X_bin_c5,Y_bin_c5,root1,'bin_c5',clf_array,1)
print("_________________________________")

#C10 1--testing
print('C10')
X_train_bin_c10,Y_train_bin_c10=getXYforfeature('bin','bin','C',root1,1,x=10)
X_bin_c10,Y_bin_c10=getXYforfeature('bin','bin','C',root1,0,x=10)
clf_array=load_models_run_test('bin_c10',root1,X_bin_c10,Y_bin_c10)
generate_thresh_files(X_bin_c10,Y_bin_c10,root1,'bin_c10',clf_array,0)
generate_thresh_files(X_bin_c10,Y_bin_c10,root1,'bin_c10',clf_array,1)
print("_________________________________")


#C15 1--testing
print('C15')
X_train_bin_c15,Y_train_bin_c15=getXYforfeature('bin','bin','C',root1,1,x=15)
X_bin_c15,Y_bin_c15=getXYforfeature('bin','bin','C',root1,0,x=15)
clf_array=load_models_run_test('bin_c15',root1,X_bin_c15,Y_bin_c15)
generate_thresh_files(X_bin_c15,Y_bin_c15,root1,'bin_c15',clf_array,0)
generate_thresh_files(X_bin_c15,Y_bin_c15,root1,'bin_c15',clf_array,1)
print("_________________________________")


#N5C5 1--testing
print('N5C5')
X_train_bin_nc5,Y_train_bin_nc5=getXYforfeature('bin','bin','NC',root1,1,x=5,y=5)
X_bin_nc5,Y_bin_nc5=getXYforfeature('bin','bin','NC',root1,0,x=5,y=5)
clf_array=load_models_run_test('bin_nc5',root1,X_bin_nc5,Y_bin_nc5)
generate_thresh_files(X_bin_nc5,Y_bin_nc5,root1,'bin_nc5',clf_array,0)
generate_thresh_files(X_bin_nc5,Y_bin_nc5,root1,'bin_nc5',clf_array,1)
print("_________________________________")




#N10C10 1--testing
print('N10C10')
X_train_bin_nc10,Y_train_bin_nc10=getXYforfeature('bin','bin','NC',root1,1,x=10,y=10)
X_bin_nc10,Y_bin_nc10=getXYforfeature('bin','bin','NC',root1,0,x=10,y=10)
clf_array=load_models_run_test('bin_nc10',root1,X_bin_nc10,Y_bin_nc10)
generate_thresh_files(X_bin_nc10,Y_bin_nc10,root1,'bin_nc10',clf_array,0)
generate_thresh_files(X_bin_nc10,Y_bin_nc10,root1,'bin_nc10',clf_array,1)
print("_________________________________")

#N15C15 1--testing
print('N15C15')
X_train_bin_nc15,Y_train_bin_nc15=getXYforfeature('bin','bin','NC',root1,1,x=15,y=15)
X_bin_nc15,Y_bin_nc15=getXYforfeature('bin','bin','NC',root1,0,x=15,y=15)
clf_array=load_models_run_test('bin_nc15',root1,X_bin_nc15,Y_bin_nc15)
generate_thresh_files(X_bin_nc15,Y_bin_nc15,root1,'bin_nc15',clf_array,0)
generate_thresh_files(X_bin_nc15,Y_bin_nc15,root1,'bin_nc15',clf_array,1)
print("_________________________________")





N5
MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)
SVC(C=2, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.5, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')
RidgeClassifier(alpha=15.0, class_weight=None, copy_X=True,
        fit_intercept=True, max_iter=None, normalize=False,
        random_state=None, solver='auto', tol=0.0001)
Rand

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=16, max_features=16, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=16, max_features=6, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=3,
           min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 20, 30), learning_rate

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')
RidgeClassifier(alpha=50.0, class_weight=None, copy_X=True,
        fit_intercept=True, max_iter=None, normalize=False,
        random_state=None, solver='auto', tol=0.0001)
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=16, max_features=11, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
ExtraTreesClassifier(bootstr

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=16, max_features=16, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
_________________________________
N10C10
MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 10, 15), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='sgd', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)
SVC(C=2, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
 